In [1]:
import json
import math
from collections import namedtuple

BEST_RPM = 1800
CONFIG_friction_drag_multiplier = 0.0001
CONFIG_aero_dynamic_drag_multiplier = 0.0001
CONFIG_fluid_per_blade_liner_kilometre = 20

Material = namedtuple('Material', 'extraction_rate induction_energy_exponent_bonus induction_efficiency name')

materials = {
    "ATM块": Material(0.4, 1.02, 1.2, 'allthemodium'),
    "难得素": Material(0.7, 1.06, 1.5, 'unobtainium'),
    "振金": Material(0.5, 1.04, 1.35, 'vibranium'),
    "镥块": Material(0.35, 1.02, 1.15, 'biggerreactors:ludicrite_block'),
    "铝块": Material(0.13, 1, 0.495, 'aluminum'),
    "铜块": Material(0.12, 1, 0.396, 'copper'),
    "琥珀金块": Material(0.2, 1, 0.825, 'electrum'),
    "末影块": Material(0.3, 1.02, 0.99, 'enderium'),
    "殷钢块": Material(0.14, 1, 0.495, 'invar'),
    "铂块": Material(0.25, 1, 0.99, 'platinum'),
    "银块": Material(0.15, 1, 0.561, 'silver'),
    "锇块": Material(0.12, 1, 0.462, 'osmium'),
    "金块": Material(0.175, 1, 0.66, 'gold'),
    "铁块": Material(0.1, 1, 0.33, 'iron'),
    "钢块": Material(0.13, 1, 0.495, 'steel'),
}


def blade_length(shaft_length, upper, coil_length, idc_):
    rotor_mass = shaft_length * 100 + (shaft_length - coil_length) * 100 * upper * 4
    linear_blade_meters_per_revolution = ((1 + upper) * upper) / 2 * 4 * (shaft_length - coil_length)
    effective_flow_rate = (BEST_RPM * idc_ + rotor_mass * math.pow(BEST_RPM * CONFIG_friction_drag_multiplier, 2) + linear_blade_meters_per_revolution * math.pow(BEST_RPM * CONFIG_aero_dynamic_drag_multiplier, 2)) / 10
    linear_blade_meters_per_revolution_cal = effective_flow_rate / BEST_RPM / (2 * math.pi) * 1000 / CONFIG_fluid_per_blade_liner_kilometre
    full_blade_length = math.ceil(linear_blade_meters_per_revolution_cal / (((1 + upper) * upper) * 2))
    return full_blade_length, effective_flow_rate, linear_blade_meters_per_revolution_cal


def coil_position(coils):
    if len(coils) <= 2:
        return coils
    else:
        return {1: coils[1], len(coils): coils[len(coils)]}


def ret(ind, shaft_length, upper, i, material, ef, coils, coil_size, max_steam, show_details, show_coils):
    full_blade_length, effective_flow_rate, _ = blade_length(shaft_length, upper, i, ind)
    energy = math.pow(BEST_RPM * ind, material.induction_energy_exponent_bonus) * material.induction_efficiency * (0.25 * math.cos(BEST_RPM / (45.5 * math.pi)) + 0.75)
    rf_mb = energy / ef
    if show_details:
        print("线圈方块总数:", coil_size)
        if show_coils:
            print("线圈方块放置方式:", json.dumps(coils, sort_keys=True, indent=4))
        print(f"蒸汽流速: {ef:.3f}")
        print(f"rf/t: {energy:.3f}")
        print(f"rf/mb: {rf_mb:.3f}")
        # print(f"total: {_:.3f}")
    return {'coil_size': coil_size, 'coil_position': coil_position(coils), 'flow_rate': round(ef, 3),
            'rf_t': round(energy, 3), 'rf_mb': round(rf_mb, 3), "status": ef <= max_steam}


def turbine(shaft_length, width, material: Material, show_details=True, show_coils=True):
    ef = 0
    coil_size = 0
    coils = {}
    max_steam = (width ** 2 - 1) * 5000
    inductor_drag_coefficient = 0

    for i in range(1, shaft_length):
        for j in range(3, width + 1, 2):
            for k in range(1, j ** 2 - (j - 2) ** 2 + 1):
                distance = (j - 1) / 2
                idc = inductor_drag_coefficient
                inductor_drag_coefficient += material.extraction_rate * (2 / (distance + 1)) * 10
                upper = (width - 1) / 2
                full_blade_length, effective_flow_rate, _ = blade_length(shaft_length, upper, i, inductor_drag_coefficient)
                if i + full_blade_length <= shaft_length:
                    ef = effective_flow_rate
                    coil_size += 1
                    if i not in coils:
                        coils[i] = {j: 1}
                    else:
                        if j not in coils[i]:
                            coils[i][j] = 1
                        else:
                            coils[i][j] += 1
                elif width > 3:
                    inductor_drag_coefficient = idc
                    for l in range(width, 3, -2):
                        for m in range(1, l ** 2 - (l - 2) ** 2 + 1):
                            distance = (l - 1) / 2
                            idd = inductor_drag_coefficient
                            inductor_drag_coefficient += material.extraction_rate * (2 / (distance + 1)) * 10
                            upper = (width - 1) / 2
                            full_blade_length, effective_flow_rate, _ = blade_length(shaft_length, upper, i, inductor_drag_coefficient)
                            if i + full_blade_length <= shaft_length:
                                ef = effective_flow_rate
                                coil_size += 1
                                if i not in coils:
                                    coils[i] = {l: 1}
                                else:
                                    if l not in coils[i]:
                                        coils[i][l] = 1
                                    else:
                                        coils[i][l] += 1
                            else:
                                return ret(idd, shaft_length, upper, i, material, ef, coils, coil_size, max_steam, show_details, show_coils)
                else:
                    return ret(idc, shaft_length, upper, i, material, ef, coils, coil_size, max_steam, show_details, show_coils)


# 一次将 14，9，"镥块" 替换成你的涡轮机内部长、宽和线圈方块
# 可以使用的线圈在上面有列表
turbine(14, 9, materials['镥块'])


sss
线圈方块总数: 256
线圈方块放置方式: {
    "1": {
        "3": 8,
        "5": 16,
        "7": 24,
        "9": 32
    },
    "2": {
        "3": 8,
        "5": 16,
        "7": 24,
        "9": 32
    },
    "3": {
        "3": 8,
        "5": 16,
        "7": 24,
        "9": 32
    },
    "4": {
        "3": 8,
        "5": 7,
        "9": 1
    }
}
蒸汽流速: 90441.672
rf/t: 1367332.778
rf/mb: 15.118
done
